In [1]:
!pip install imblearn

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

EPOCHS = 30
BATCH_SIZE = 64

transform = transforms.Compose([
    transforms.ToTensor()
])

data=pd.read_excel('./modeling_data/result2.xlsx')

train_x=data.iloc[:1352,1:-1]
test_x=data.iloc[1352:,1:-1]

train_y=data.iloc[:1352,-1:]
test_y=data.iloc[1352:,-1:]

train_x

,1,2,3,4,5,6,7
0,45.000000,20.544193,72.000000,0.0,0.000000,0.00000,148.000000
1,45.000000,20.544193,72.000000,0.0,0.000000,0.00000,148.000000
2,45.000000,20.544193,72.000000,0.0,0.000000,0.00000,148.000000
3,45.000000,20.544193,72.000000,0.0,0.000000,0.00000,148.000000
4,45.000000,20.544193,72.000000,0.0,0.000000,0.00000,148.000000
...,...,...,...,...,...,...,...
1347,29.000000,14.483887,35.000000,0.0,0.000000,0.00000,141.500000
1348,29.000000,14.483887,35.000000,0.0,0.000000,0.00000,141.500000
1349,32.739286,16.471676,8002.909091,0.0,1824.340336,443.42236,141.028571
1350,32.479464,16.319777,8002.909091,0.0,1824.340336,443.42236,141.114286


In [3]:
train_y_seires=train_y.squeeze()
type(train_y_seires)

pandas.core.series.Series

In [4]:
print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(train_y_seires).value_counts())

SMOTE 적용 전 레이블 값 분포: 
  0    1275
 1      40
-1      37
Name: Label, dtype: int64


In [5]:
minMaxScaler = MinMaxScaler()
minMaxScaler.fit(train_x) 
train_data_minMaxScaled = minMaxScaler.transform(train_x)

train_data_minMaxScaled

smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(train_data_minMaxScaled, train_y)

type(y_train_over)

train_data_minMaxScaled.shape

(1352, 7)

In [6]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()	# 필수
        self.w1 = nn.Linear(7, 64)	# Linear한 1차원 값의 행렬곱을 할 때 사용 한다.
        self.w2 = nn.Linear(64, 1)  # y = xA + b, bias 값을 가지고 있다.bias 비활성화를 원하면 bias=False를 파라미터에 입력 하면 된다.
        self.relu = nn.ReLU()	# ReLU 활성화 함수
        self.softmax = nn.Softmax(dim=0)	# Softmax 함수
        self.dropout = nn.Dropout(0.5)	# Dropout을 위함}
        bias=False
    def forward(self, x):	# 값을 도출하는 함수
        y = self.w1(x) 
        y = self.relu(y)
        y = self.dropout(y)
        y = self.w2(y) 
        y = self.softmax(y)
        return y


In [7]:
torch_tensor_tr = torch.Tensor(X_train_over).float()
torch_tensor_ts = torch.Tensor(test_x.values).float()

train_y_new=y_train_over.to_numpy()
test_y_new=test_y.to_numpy()

train_y_new_tensor = torch.DoubleTensor(train_y_new)
test_y_new_tensor = torch.DoubleTensor(test_y_new)


In [8]:
train_y_new_tensor

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]], dtype=torch.float64)

In [9]:
model = DNN() # 객체 선언
criterion = torch.nn.CrossEntropyLoss()  # loss function 설정
optim = torch.optim.Adam(model.parameters(), lr=0.001) # Optimizer 설정


In [10]:
for epoch in range(1000):
    output = model(torch_tensor_tr)	# model의 forward 함수 호출  
    loss = criterion(output, train_y_new_tensor)	# loss function으로 값 계산

    optim.zero_grad()	# 변화도를 0으로 만듦
    loss.backward() # 역전파
    optim.step()	# 역전파로 알아낸 변화도를 model에 적용

    print("progress:", epoch, "loss=", loss.item())

progress: 0 loss= -0.0
progress: 1 loss= -0.0
progress: 2 loss= -0.0
progress: 3 loss= -0.0
progress: 4 loss= -0.0
progress: 5 loss= -0.0
progress: 6 loss= -0.0
progress: 7 loss= -0.0
progress: 8 loss= -0.0
progress: 9 loss= -0.0
progress: 10 loss= -0.0
progress: 11 loss= -0.0
progress: 12 loss= -0.0
progress: 13 loss= -0.0
progress: 14 loss= -0.0
progress: 15 loss= -0.0
progress: 16 loss= -0.0
progress: 17 loss= -0.0
progress: 18 loss= -0.0
progress: 19 loss= -0.0
progress: 20 loss= -0.0
progress: 21 loss= -0.0
progress: 22 loss= -0.0
progress: 23 loss= -0.0
progress: 24 loss= -0.0
progress: 25 loss= -0.0
progress: 26 loss= -0.0
progress: 27 loss= -0.0
progress: 28 loss= -0.0
progress: 29 loss= -0.0
progress: 30 loss= -0.0
progress: 31 loss= -0.0
progress: 32 loss= -0.0
progress: 33 loss= -0.0
progress: 34 loss= -0.0
progress: 35 loss= -0.0
progress: 36 loss= -0.0
progress: 37 loss= -0.0
progress: 38 loss= -0.0
progress: 39 loss= -0.0
progress: 40 loss= -0.0
progress: 41 loss= -0.0
pr

progress: 362 loss= -0.0
progress: 363 loss= -0.0
progress: 364 loss= -0.0
progress: 365 loss= -0.0
progress: 366 loss= -0.0
progress: 367 loss= -0.0
progress: 368 loss= -0.0
progress: 369 loss= -0.0
progress: 370 loss= -0.0
progress: 371 loss= -0.0
progress: 372 loss= -0.0
progress: 373 loss= -0.0
progress: 374 loss= -0.0
progress: 375 loss= -0.0
progress: 376 loss= -0.0
progress: 377 loss= -0.0
progress: 378 loss= -0.0
progress: 379 loss= -0.0
progress: 380 loss= -0.0
progress: 381 loss= -0.0
progress: 382 loss= -0.0
progress: 383 loss= -0.0
progress: 384 loss= -0.0
progress: 385 loss= -0.0
progress: 386 loss= -0.0
progress: 387 loss= -0.0
progress: 388 loss= -0.0
progress: 389 loss= -0.0
progress: 390 loss= -0.0
progress: 391 loss= -0.0
progress: 392 loss= -0.0
progress: 393 loss= -0.0
progress: 394 loss= -0.0
progress: 395 loss= -0.0
progress: 396 loss= -0.0
progress: 397 loss= -0.0
progress: 398 loss= -0.0
progress: 399 loss= -0.0
progress: 400 loss= -0.0
progress: 401 loss= -0.0


progress: 694 loss= -0.0
progress: 695 loss= -0.0
progress: 696 loss= -0.0
progress: 697 loss= -0.0
progress: 698 loss= -0.0
progress: 699 loss= -0.0
progress: 700 loss= -0.0
progress: 701 loss= -0.0
progress: 702 loss= -0.0
progress: 703 loss= -0.0
progress: 704 loss= -0.0
progress: 705 loss= -0.0
progress: 706 loss= -0.0
progress: 707 loss= -0.0
progress: 708 loss= -0.0
progress: 709 loss= -0.0
progress: 710 loss= -0.0
progress: 711 loss= -0.0
progress: 712 loss= -0.0
progress: 713 loss= -0.0
progress: 714 loss= -0.0
progress: 715 loss= -0.0
progress: 716 loss= -0.0
progress: 717 loss= -0.0
progress: 718 loss= -0.0
progress: 719 loss= -0.0
progress: 720 loss= -0.0
progress: 721 loss= -0.0
progress: 722 loss= -0.0
progress: 723 loss= -0.0
progress: 724 loss= -0.0
progress: 725 loss= -0.0
progress: 726 loss= -0.0
progress: 727 loss= -0.0
progress: 728 loss= -0.0
progress: 729 loss= -0.0
progress: 730 loss= -0.0
progress: 731 loss= -0.0
progress: 732 loss= -0.0
progress: 733 loss= -0.0


In [ ]:
for x in torch_tensor_tr:    
    y_pred = model(x)	# 결과
    print(y_pred.max(dim=0)[1].item()) # Softmax로 나온 최고 값의 Item 반환

In [ ]:
type(test_x)

In [ ]:
torch_tensor_tr_y.reshape(:,1)

train_y_new=train_y.to_numpy()

train_y_new=train_y_new.reshape(-1)

train_y_new

## 2번

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


In [ ]:
data=pd.read_csv('./modeling_data/2weeks_summary2.csv')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

    def forward(self, x):
        x = x.float()
        h1 = F.relu(self.fc1(x.view(-1, 784)))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        h6 = self.fc6(h5)
        return F.log_softmax(h6, dim=1)

print("init model done")

In [ ]:
# train_x=data.iloc[:1352,1:-1]
# test_x=data.iloc[1352:,1:-1]

# train_y=data.iloc[:1352,-1:]
# test_y=data.iloc[1352:,-1:]
x=data.iloc[:,1:-1]
y=data.iloc[:,-1:]

In [ ]:
torch_tensor_x = torch.tensor(x.values)
torch_tensor_y = torch.tensor(y.values)


In [ ]:
torch_tensor_y

In [ ]:
# dataset = list(zip(x, y)) # zip 함수 사용 예시
dataset = TensorDataset(torch_tensor_x, torch_tensor_y) # TensorDataset 함수 사용 예시

In [ ]:
dataset

In [ ]:

batch_size = 1
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print("set vars and device done")

In [ ]:
train_loader = torch.utils.data.DataLoader(
  dataset, batch_size = batch_size, shuffle=True
)

In [ ]:
test_loader = torch.utils.data.DataLoader(
        (dataset, train=False, download=True,
                 transform=transform), 
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [ ]:
for batch in train_loader:
    batch_x = batch[0] # 분할된 32개의 x 벡터 모음
    batch_y = batch[1] # 분할된 32개의 y 벡터 모음(x 벡터와 pair 순서 일치)

In [ ]:
batch_y.shape